In [1]:
from pyspark.sql import *
from delta import *
from pyspark.sql.functions import split, to_date, hour, minute, second


builder = SparkSession.builder.appName("partition").config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension").config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
spark = configure_spark_with_delta_pip(builder).getOrCreate()

INPUT_PATH = "/home/felipe/code/teste_particionamento/dados/"
BRONZE_PATH = "/home/felipe/code/teste_particionamento/lake/bronze/"
SILVER_PATH = "/home/felipe/code/teste_particionamento/lake/silver/"

# df = spark.read.format("parquet").option("inferSchema","true").option("header","true").load(f"{INPUT_PATH}/sp_micro/")
# df = df.withColumn("data", to_date("tempo_captura")).withColumn("hora", split(split("tempo_captura", "T")[1], "Z")[0])



# spark.stop()

24/03/05 10:11:03 WARN Utils: Your hostname, desktop resolves to a loopback address: 127.0.1.1; using 192.168.0.104 instead (on interface enp6s0)
24/03/05 10:11:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/felipe/.local/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/felipe/.ivy2/cache
The jars for the packages stored in: /home/felipe/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8ecfbd82-1005-41fc-95c0-6c6a2d2c2a97;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 137ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0 

In [6]:
from pyspark.sql import *
from delta import *
from pyspark.sql.functions import split, to_date


builder = SparkSession.builder.appName("topicos").config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension").config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
spark = configure_spark_with_delta_pip(builder).getOrCreate()

INPUT_PATH = "/home/felipe/code/teste_particionamento/dados/"
BRONZE_PATH = "/home/felipe/code/teste_particionamento/lake/bronze"
SILVER_PATH = "/home/felipe/code/teste_particionamento/lake/silver"


# Lendo os arquivos Parquet
df = spark.read.format("parquet").option("inferSchema","true").option("header","true").load(f"{INPUT_PATH}/sp_micro/")

# Criando a coluna "data" e "hora" para realizarmos a partição por data
df = df.withColumn("data", to_date("tempo_captura")).withColumn("hora", split(split("tempo_captura", "T")[1], "Z")[0])

# Criando o particionamento
df.write.mode("overwrite").partitionBy("data").format("delta").saveAsTable("bronze_sp2")



In [23]:
import levi
import deltalake
from datetime import datetime

dt = deltalake.DeltaTable(f"/home/felipe/code/teste_particionamento/spark-warehouse/bronze_sp2", version=0)
levi.delta_file_sizes(dt)


{'num_files_<1mb': 16,
 'num_files_1mb-500mb': 0,
 'num_files_500mb-1gb': 0,
 'num_files_1gb-2gb': 0,
 'num_files_>2gb': 0}

In [24]:
levi.skipped_stats(dt, filters=[("tempo_captura", "=", "'2024-02-13T21:56:05Z'")])

{'num_files': 16, 'num_files_skipped': 12, 'num_bytes_skipped': 2328675}

In [7]:

df = spark.read.format("delta").table("bronze_sp2")
df.show()



+-------------------+-------------------+--------------------+---------+-------+-------------------+--------------------+----------+--------+
|           latitude|          longitude|       tempo_captura|id_onibus|      c|                lt0|                 lt1|      data|    hora|
+-------------------+-------------------+--------------------+---------+-------+-------------------+--------------------+----------+--------+
|       -23.65332575|       -46.74067475|2024-01-31T02:13:52Z|    71212|609F-10|TERM. PRINC. ISABEL|       CHÁC. SANTANA|2024-01-31|02:13:52|
|         -23.534974|        -46.6443435|2024-01-31T02:14:13Z|    71872|609F-10|TERM. PRINC. ISABEL|       CHÁC. SANTANA|2024-01-31|02:14:13|
|        -23.5953105|-46.695625500000006|2024-01-31T02:14:13Z|    71740|609F-10|TERM. PRINC. ISABEL|       CHÁC. SANTANA|2024-01-31|02:14:13|
|        -23.5706045|        -46.6626015|2024-01-31T02:13:31Z|    71754|609F-10|TERM. PRINC. ISABEL|       CHÁC. SANTANA|2024-01-31|02:13:31|
|     

In [19]:
(
    spark.read.format("delta").table("bronze_sp2").createOrReplaceTempView("particao")
)

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `bronze_sp2` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.;
'UnresolvedRelation [bronze_sp2], [], false


In [18]:
%%time 

spark.sql("select latitude, longitude from particao where tempo_captura = '2024-02-13T21:56:05Z'")

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `particao` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 32;
'Project ['latitude, 'longitude]
+- 'Filter ('tempo_captura = 2024-02-13T21:56:05Z)
   +- 'UnresolvedRelation [particao], [], false


In [2]:
(
    spark.read.format("delta").option("versionAsOf", "0").load("/home/felipe/code/teste_particionamento/spark-warehouse/bronze_sp2").createOrReplaceTempView("v0")
)

In [5]:
%%time 

spark.sql("select latitude, longitude from v0 where data = '2024-02-13'")

CPU times: user 605 µs, sys: 1.1 ms, total: 1.7 ms
Wall time: 11.6 ms


DataFrame[latitude: double, longitude: double]